# CS 109A/AC 209A/STAT 121A Data Science: Airbnb Project
## Build an Improved Model
**Harvard University**  
**Fall 2016**  
**Team**: Stephen Camera-Murray, Himani Garg, and Vijay Thangella  
**TF**: Christine Hwang

### IV. Build an Improved Model

Import libraries

In [2]:
# import libraries
import warnings
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import random
from sklearn.linear_model import LogisticRegression

# suppress warnings
warnings.filterwarnings ( 'ignore' )
%matplotlib inline

We begin by loading the listings dataset we saved after cleansing from milestone #4:  
- cleansed_listings.csv.gz - the New York City Airbnb listing data from January 2015 (cleansed)

In [3]:
# load listings data into a pandas df
listingsDF = pd.read_csv (  './datasets/cleansed_listings_with_target.tab.gz', sep = '\t', index_col = 'id', compression = 'gzip' )

# display the first two rows
listingsDF.head ( n = 2 )

,name,neighborhood,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,square_feet,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,host_listing_count,months_as_host,months_since_first_review,months_since_last_review,price_category
id,,,,,,,,,,,,,,,,,,,,,
1069266,Stay like a real New Yorker!,Midtown East,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,1.0,...,7.0,9.0,9.0,10.0,9.0,1,21,21,1,Mid
2061725,Option of 2 Beds w Private Bathroom,Bushwick,Apartment,Private room,2,1.0,1.0,2.0,Real Bed,1.0,...,10.0,10.0,10.0,9.0,10.0,4,24,11,1,Low


In [4]:
listingsDF.shape

(19526, 30)

#### Check for Collinearity
Next, we’ll create scatter matrices for any variables we might suspect to have some collinearity since doing them all would be unrealistic. If we see any very obvious relationships, we’ll drop one of the variables.

#### Build an Improved Model
Next, we’ll try a few different models with their default parameters to see if any of them look promising to improve the accuracy over the baseline using F-Score as the benchmark. We’ll start with: Logistic Regression, LDA, QDA, SVM, and Random Forest. From here, we’ll select a promising model and perform grid search cross-validation to tune the model’s hyper-parameters.


#### Analyze and Present the Results
We’ll conclude with a visualization of feature importance (from the random forest model, assuming we have a reasonable fit to the data) to highlight which factors contribute to price. We’ll also include a visualization demonstrating goodness of fit (e.g. an ROC curve, suggestions welcome!). For our website, we also plan to include a static or somewhat staged demonstration of how our model could be used in the Airbnb UI.